# Geocube SDK Tutorial

-------

#### Short description

This notebook introduces you to the SDK framework with a complete Geocube workflow using the Python Client. You will see a typical workflow and how to parallelize an image processing algorithm.
This notebook is in two chapters. The first presents the SDK, the second gives an example of a workflow.

-------

#### Requirements

- Python 3.7
- The Geocube Python Client library : https://github.com/airbusgeo/geocube-client-python.git
- The url of a [Geocube Server](https://github.com/airbusgeo/geocube.git) & its Client ApiKey (for the purpose of this notebook, `GEOCUBE_SERVER` and `GEOCUBE_CLIENTAPIKEY` environment variable) - [Installation](https://github.com/airbusgeo/geocube/blob/main/INSTALL.MD) 
- The url of a [Geocube Downloader](https://github.com/airbusgeo/geocube/blob/main/INSTALL.MD#Downloader) (for the purpose of this notebook, `GEOCUBE_DOWNLOADER` environment variable)

-------

#### Table of content

- [Part 1 - SDK](#SDK)
  * [Connection Parameters](#Connection)
  * [Downloader Service](#Downloader-Service)
  * [Catalogue Functions](#Catalogue-functions)
  * [Multiprocess and Dask](#Multiprocess-and-Dask)
  * [XArray and Collection](#XArray-and-Collection)
  
  
- [Part 2 - Geocube Workflow by example: Abnormal change detection](Geocube-Client-SDK-2.ipynb#Table-of-content) (Geocube-Client-SDK-2.ipynb)


#### Notebook initialisation

In [ ]:
from datetime import datetime
from shapely import geometry
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os

from geocube import utils, entities, Consolidater, sdk

# Define the connection to the server
secure = False # in local, or true to use TLS
geocube_client_server  = os.environ['GEOCUBE_SERVER']        # e.g. 127.0.0.1:8080 for local use
geocube_client_api_key = os.environ['GEOCUBE_CLIENTAPIKEY']  # Usually empty for local use
geocube_downloader_server = os.environ['GEOCUBE_DOWNLOADER']


# SDK
Geocube Client Python provides several functions to easily scale-up an image processing pipeline.
In particular, Geocube implements an [xarray](https://xarray.pydata.org/en/stable/) backend to access geocube images using the standard `xarray.Dataset`.

As scaling-up a pipeline implies parallel computing, the Geocube entities are picklable to be transferred between processes.

### Connection
A `geocube.Client` is not picklable. `geocube.sdk` provides a convenient way to pass connection parameters: `sdk.ConnectionParams`. This function takes the same parameters as `geocube.Client` and has a function `new_client()` to connect to the server.

In [ ]:
# Define the parameters to connect to the Geocube server
connection_params = sdk.ConnectionParams(geocube_client_server, secure, geocube_client_api_key)

# Connect to the server
client = connection_params.new_client()
print("Connected to server: ", client.version())

### Downloader Service
Because the Geocube Server can be easily overwhelmed by a lot of connections, two solutions are possible to massively download data:
- deploy several Geocube servers
- run (locally or remotely) a lighter service that is in charge of downloading the images using metadata returned by the Geocube Server.

The latter case is handled by the [Geocube Downloader service](https://github.com/airbusgeo/geocube/blob/main/cmd/downloader). It can be run as a docker:
```bash
export STORAGE=[...]
docker run --rm -p 127.0.0.1:8081:8081/tcp -v $STORAGE:$STORAGE geocube-downloader -local -port 8081
```
Don't forget to mount all the local storage folders and give the proper access rights to the remote storages.
More details on the [Geocube INSTALL.MD](https://github.com/airbusgeo/geocube/blob/main/INSTALL.MD#downloader).

Downloader Service needs `CubeMetadata` to request a cube. It can be retrieved using `get_cube_metadata()` function.

In [ ]:
cube_params = entities.CubeParams.from_tile(
    tile          = entities.Tile.from_bbox((573440., 6184960.,  593920., 6205440), crs=32632, resolution=20),
    instance      = client.variable("RGB").instance("master"),
    tags          = {"source":"tutorial"},
    from_time     = datetime(2019, 1, 1),
    to_time       = datetime(2019, 5, 1),
)

In [ ]:
from geocube import Downloader
downloader = Downloader(geocube_downloader_server)

# Request metadata
metadata = client.get_cube_metadata(cube_params)

# Download the cube described by metadata
images, _ = downloader.get_cube(metadata)

print("Finished !")

#### Metadata
`CubeMetadata` and `CubeMetadata.slices` contains all that is necessary to download and format the cube and 2d-slices of data : the underlying files, their internal data format, the mapping to `CubeMetadata.ref_dformat`, the records, the transform and the crs, the resampling algorithm...

It can be used to have direct access to the file or change the resampling algorithm.

In [ ]:
for i, s in enumerate(metadata.slices):
    print(f"Image {i}:")
    for file_metadata in s.metadata:
        print(f"   - file {file_metadata.container_uri}, subdir {file_metadata.container_subdir}.")

A client can be linked to a downloader and the former will automatically use the latter to download a cube of data.

In [ ]:
client.use_downloader(downloader)

# Or using ConnectionParams
connection_params = sdk.ConnectionParams(geocube_client_server, secure, geocube_client_api_key,
                                         downloader=sdk.ConnectionParams(geocube_downloader_server))
client = connection_params.new_client()

_ = client.get_cube(cube_params, verbose=True)

### Catalogue functions
`sdk.get_cube()` is a convenient function to process a cube of data in a parallel workflow.
It takes `ConnectionParams` and `CubeParams`, downloads the cube and process it.

Two callback functions can be passed as parameter of this function:
- `image_callback` will be called for every image received by `get_cube` (see `sdk.image_callback_t`). It has to return an image  and takes as parameters:
  * `image`
  * `grouped_records` (optional)
  * `crs` or `projection`  (optional)
  * `transform` (optional)
- `cube_callback` will be called with the results of a `get_cube` (see `sdk.cube_callback_t`). The result of this function will be the returned value of `get_cube()`. It takes as parameters:
  * `timeseries` or `cube`
  * `grouped_records` (optional)
  * `crs` or `projection` (optional)
  * `transform` (optional): geotransform

All these fields will be automatically provided by `get_cube()` if they are defined.

`sdk.get_cube` is equivalent to:
```python
def sdk.get_cube():
    for image in client.get_cube():
        image = image_callback(image, [grouped_records, crs, projection, transform])
        cube.append(image)
    return cube_allback(cube, [grouped_records, crs, projection, transform])
```

In [ ]:
import numpy as np
from geocube import utils
import functools

def mean(timeseries, projection, transform, instance):
    """ Compute the mean of a timeseries over time and save it as a GeoTiff"""
    if len(timeseries) != 0:
        m = np.nanmean(timeseries, axis=-1)
        filename="outputs/" + ("_".join([f"{v:.2f}" for v in transform.to_gdal()]))+".tiff"
        utils.image_to_geotiff(m, transform, projection, instance.dformat.no_data, filename)
        return filename
    return None

# Download cube and call mean().
sdk.get_cube(connection_params, cube_params, cube_callback=functools.partial(mean, instance=client.variable("RGB").instance("master")))



### Multiprocess and Dask
The `sdk.get_cube()` function can be used for parallel processing as well as any picklable user-defined function.
With `dask.delayed`, a pool of tasks can be easily created:

In [ ]:
import dask
import functools

# Define the AOI, the records and the instance
aoi = utils.read_aoi('inputs/Denmark.json')
records = client.list_records(aoi=aoi, tags={"source":"tutorial"})
instance = client.variable("RGB").instance("master")

# Tile the AOI and create the pool of tasks
tiles = client.tile_aoi(aoi, resolution=20, crs="epsg:32632", shape=(1024,1024))
tasks = [dask.delayed(sdk.get_cube)(
    connection_params=connection_params,
    cube_params=entities.CubeParams.from_tile(t, records=records, instance=instance),
    cube_callback=functools.partial(mean, instance=instance))
         for t in tiles]


plt.rcParams['figure.figsize'] = [20, 16]
print(f"Plot {len(tiles)} tiles...")
entities.Tile.plot(tiles)

The tasks can be computed using `dask.scheduler`:

In [ ]:
dask.config.set(scheduler='synchronous')
dask.compute(tasks)
print("done")


Or with `dask.distributed`
```shell
python -m pip install "dask[distributed]"
```

In [ ]:
from dask.distributed import Client, wait
from tqdm import tqdm
import time
c = None

try:
    c = Client(n_workers=4)
    futures = c.compute(tasks)

    with tqdm(total=len(futures)) as pbar:
        while len(futures) > 0:
            nf = []
            for f in futures:
                if f.status=="finished":
                    pbar.update(1)
                else:
                    nf.append(f)
                    if f.status == "error":
                        print("retry", f)
                        f.retry()
            time.sleep(0.5)
            futures = nf
    print("done")
finally:
    if c is not None:
        c.close()
    


### Dask on GCP

reference: https://cloudprovider.dask.org/en/latest/gcp.html

#### Installation

``` bash
pip install dask-cloudprovider[gcp]>=2022.3.3 Jinja2>=3
```

Dask runs a docker on gcp workers. To build a docker embedding dask, geocube-client-python and a downloader client, run the following command in the <geocube-client-python> root folder:
``` bash
docker build -f docker/Dockerfile.dask-downloader -t dask-downloader:<tag> .
```
The downloader client is available with the following configuration: 
```python
sdk.ConnectionParams("localhost:8083")
```

Feel free to edit the dockerfile with your own librairies or use the dask-standard environnement variables `EXTRA_APT_PACKAGES` and `EXTRA_PIP_PACKAGES` (add `docker_args="-e EXTRA_PIP_PACKAGES "... ..."` when creating the cluster)
    
    
#### Configuration 

In `~/.config/dask`, you can create or edit two configuration files:

- **cloudprovider.yaml**: where it is possible to configure the image used, the type of VM, the GCP project etc. The elements defined in this file will be the default values when instantiating a GCPCluster object.

Ex:
``` yaml
cloudprovider:
  gcp:
    source_image: "projects/ubuntu-os-cloud/global/images/ubuntu-minimal-2004-focal-v20220419a" # the gcp image to use for all instances (`gcloud compute images list`)
    zone: "<zone-id>" # the zone of where to launch the instances (eg: europe-west1-d)
    network: "<subnet>" # the network/subnetwork in GCP to use
    network_projectid: null # GCP project id where the network exists
    projectid: "<project-id>" # name of the google cloud project
    on_host_maintenance: "TERMINATE"
    machine_type: "n1-standard-16" # size of the machine type to use (`gcloud compute machine-types list`)
    filesystem_size: 50 # amount in GBs of hard drive space to allocate
    ngpus: "" # number of GPUs to use
    gpu_type: "" # type of gpus to use: nvidia-tesla-k80, nvidia-tesla-p100, nvidia-tesla-t4
    disk_type: "pd-ssd" # type of disk to use: pd-standard, pd-ssd
    docker_image: "<image>:<tag>" # docker image to use (eg. eu.gcr.io/<project-id>/<image>:<tag>)
    auto_shutdown: true # Shutdown instances automatically if the scheduler or worker services time out.
    preemptible: true  # Whether to use preemptible instances for workers
    public_ingress: true # configure the scheduler to be externally accessible.  This assumes firefwall rules for 8787 and 8786
    extra_bootstrap: ["gcloud auth print-access-token | docker login -u oauth2accesstoken --password-stdin https://eu.gcr.io"] # If the image is private and stored in the gcp registry

```


- **debug.yaml**: where it is possible to define logging strategy

Ex:

``` yaml

logging:
  version: 1
  handlers:
    file:
      class: logging.handlers.RotatingFileHandler
      filename: output.log
      level: INFO
    console:
      class: logging.StreamHandler
      level: INFO
  loggers:
    distributed.worker:
      level: INFO
      handlers:
        - file
        - console
    distributed.scheduler:
      level: INFO
      handlers:
        - file
        - console
```

#### Prerequisite

- Build and push the dask-downloader docker image
- Grant the appropriate rights to the default service account for pulling the docker image.
- Set your GCP authentication credentials.

#### Create GCPCluster

1. A scheduler VM is created
2. The dask-downloader docker image is pulled and dask-scheduler is started
3. Multiple Workers VM are created (n_workers=number of VM)

NB: 
- It takes a couple of minutes to create the scheduler and the workers. Be patient ! Connect to the VM and look at the logs if you think it takes longer than usual.
```shell
gcloud compute ssh --project=<PROJECT-ID> --zone <ZONE> <dask-########-scheduler> --command "tail /var/log/cloud-init-output.log -n 100 -f"
```
- If the docker image is stored on a private container registry, you need to log docker using extra_boostrap. (see: https://cloudprovider.dask.org/en/latest/troubleshooting.html#pulling-private-docker-images)
- By default, the scheduler runs on 8786 port and the dashboard is viewable on 8787. If these ports are blocked, they can be override using for example `scheduler_options={"port":"8080", "dashboard_address":"80"}`.
- **Don't forget to close the cluster with `cluster.close()` or manually delete the vm in case of failure**

In [ ]:
from dask_cloudprovider.gcp import GCPCluster
from dask.distributed import Client

# Create Cluster
cluster = GCPCluster(n_workers=1, scheduler_options={"port":"8080"}, docker_args="-v /var/log/container:/var/log")

# Create client
gcpclient = Client(cluster)

To connect to the downloader embedded in the dask worker, use `sdk.ConnectionParams("127.0.0.1:8083")`. It will be faster than using the geocube to download the images.

In [ ]:
import dask

@dask.delayed
def get_cube(connection_params, cube_params):
    connection_params.downloader = sdk.ConnectionParams("127.0.0.1:8083")
    client = connection_params.new_client()
    return client.get_cube(cube_params, verbose=True)

images, records = get_cube(connection_params, cube_params).compute()
  


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(images[0][...,0])


Don't forget to close the client and the cluster to delete the virtual machines.

In [ ]:
gcpclient.close()
cluster.close()

#### Logging and debug

Logs are available :
- in the VM: `/var/log/cloud-init-output.log` and `/var/log/cloud-init.log`
- in the container: `/var/log/downloader_output.log` and `/var/log/python_output.log` or directly in the `/var/log/container` folder of the VM with `docker_args="-v /var/log/container:/var/log"` argument when creating `GCPCluster` .


`gcloud compute ssh --project=<PROJECT-ID> --zone <ZONE> <dask-########-worker-########> --command "tail /var/log/container/python_output.log -n 100 -f"`

With `debug=True`, GCPCluster prints some logs:

#### Example

In [ ]:
cluster = GCPCluster(n_workers=1, debug=True, docker_args="-v /var/log/container:/var/log")
cluster.close()

### XArray and Collection
A `Collection` describes a collection of datasets corresponding to several variables and a set of records.
It is readable by `xarray`:

In [ ]:
import xarray
from geocube import sdk, entities
import matplotlib.pyplot as plt
import math

# Create a tile
record = client.list_records("S2B_MSIL1C_20190105T103429_N0207_R108_T32UNG_20190105T122413", with_aoi=True)[0]
tile = entities.Tile.from_record(record=record, crs="epsg:32632", resolution=20)

# Select variables
instances = [
    client.variable("RGB").instance("master"),
    client.variable("NDVI").instance("master")
]

# Select records
records = client.list_records(tags={'source':'tutorial','constellation':'SENTINEL2'})

# Create collection
collection = sdk.Collection.from_tile(tile, records=records, instances=instances)

# Open collection
#ds = xarray.open_dataset(collection, connection_params=connection_params, block_size=(256, 256), engine=sdk.GeocubeBackendEntrypoint)
ds = sdk.open_geocube(collection, connection_params=connection_params, block_size=(256, 256))

# Get timeseries of RGB
images = ds["RGB:master"][0:1000,3000:4000].compute()


In [ ]:

nbimages=images.shape[3]
plt.rcParams['figure.figsize'] = [20, 16]
for f in range(0, nbimages):
    plt.subplot(math.ceil(nbimages/4), 4, f+1).set_axis_off()
    plt.imshow(images[:,:,:, f]/255)

# Next step: demonstration of a full workflow by example
In the [next notebook](Geocube-Client-SDK-2.ipynb), the sdk will be used to address a simple but typical use-case: abnormal change detection.